### Assignment - Segmenting and Clustering Neighborhoods in Toronto - Answer - 2

In [2]:
#Install necessary packages
!conda install -c anaconda lxml --yes
!conda install -c conda-forge beautifulsoup4 --yes #Install Beautifulsoup version 4

In [3]:
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

In [4]:
# Restart kernel for new installations to take effect
restartkernel()

# Import necessary libraries/ packages
import pandas as pd
import numpy as np
import requests
import lxml
from bs4 import BeautifulSoup #Import the library

print('Beautiful Soup Installed!!')

Beautiful Soup Installed!!


In [5]:
# Read file from the url
html_file = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

# Parse HTML file
soup = BeautifulSoup(html_file.content, 'lxml')

In [6]:
# Read Table tag as string 
tables = soup.find_all('table')

# Parse data in HTML tag to a list 
df_list = pd.read_html(str(tables))

# Convert list of DataFrame object to a dataframe
df = pd.DataFrame(df_list[0])

In [7]:
# Drop all rows where Borough is not assigned
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)

# Using Numpy update all neighbourhood to same as Borough if the value is "Not Assigned"
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'], df['Neighbourhood'])

# Reset index as some rows were deleted in earlier steps
df.reset_index(drop=True, inplace=True)

df.sort_values(by=['Postcode'], inplace=True)

df = df.groupby(["Postcode", "Borough"])["Neighbourhood"].agg(lambda x: ','.join(x.astype(str))).reset_index()
print(df.shape)

(103, 3)


### Geocoder was not responding so using CSV file provided with the assignment

In [8]:
# Get the CSV file containing postal code
!wget -q -O 'canada_postal_code.csv' http://cocl.us/Geospatial_data

# Read CSV file into a dataframe
postal_code_df = pd.read_csv('canada_postal_code.csv')

In [9]:
final_df = pd.merge(df, postal_code_df, left_on='Postcode', right_on='Postal Code')
final_df.drop('Postal Code', axis=1, inplace=True)
final_df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile,Oakridge,Clairlea",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Scarborough Village West,Cliffside",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West,Birch Cliff",43.692657,-79.264848
